<a href="https://colab.research.google.com/github/eddes/buildingphysics/blob/feature%2Fpipenv_and_notebooks/notebooks/chapter_2/PCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase change materials

In this notebook, we expose a numerical implementation for conduction in materials including phase
change material (PCM), using the apparent thermal capacity method, with a slight increment in complexity
compared to the examples of Chapter 1.


In [33]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
rouge_A='#C60C2E'
vert1_A='#005157'
vert2_A='#627D77'
vert3_A='#9EB28F'
vert4_A='#C5E5A4'
medblue='royalblue'
gris1_A='#595A5C'

colors=[rouge_A, vert1_A, vert2_A, vert3_A, vert4_A, medblue, gris1_A]


## Helper functions
We define some method to model the phase material change.
- `fc_fraction` returns the fraction of liquid `f` given current temperature `T`, fusion temperature `Tf` and the fusion interval `dTf`.
- `fc_distrib` returns the distribution function for the apparent capacity 
- `fc_Cp_apparent` return the apparent thermal capacity of the material

In [39]:

def fc_fraction(T,dTf,Tf):
    if   T<(Tf-2*dTf): f=0 # all solid
    elif T>(Tf+2*dTf): f=1 # all liquid
    else:f=(T-Tf+2*dTf)/(dTf*4)
    return f

def fc_distrib(T,dTf,Tf):
    T=T+273.15
    Tf=Tf+273.15
    return np.exp(-((T-Tf)/(dTf))**2)/np.sqrt(np.pi*dTf**2)

def fc_Cp_apparent(T,dTf,Tf,Lf,Cp_s,Cp_l):
    fraction=fc_fraction(T,dTf,Tf)
    distrib=fc_distrib(T,dTf,Tf)
    return Lf*distrib + Cp_s + fraction*(Cp_l-Cp_s)


Defining here the K matrix as parti of the explicit Euler scheme (cf chapter 1).

In [42]:
def init_K(n):
    K=np.eye(n,n,k=-1)*1 + np.eye(n,n)*-2 + np.eye(n,n,k=1)*1
    K[0,0]=0
    K[0,1]=0
    K[-1,-1]=0
    K[-1,-2]=0
    return K

### Simulation and plot definition
A method initializing Fo for each cell of the simulation is definied.


In [43]:
#intialize the local Fourier number for PCMs
def generate_Fo(n,dt,T_init,dTf,Tf):
    Fo=np.zeros(n)
    for i in range(len(Fo)):
        Fo[i]=k*dt/(rho*fc_Cp_apparent(T_init,dTf,Tf,Lf,Cp_s,Cp_l)*dx**2)
        # local stability check
        if Fo[i]>0.5:
            print("Fo - stability issue... i=",i)
            dt_min=0.5*dx**2*rho*Cp_s/k
            print("minimum time step =", round(dt_min,2))
            dt=0.9*dt_min
            print("changing to =", round(dt,2))
    return Fo

In [44]:
def compute_and_plot_PCM(Tf, dTf):
    
    # Cps in J/kg/K
    Cp_s=1800  # the sensible heat of the solid
    Cp_l=2400  # the sensible heat of the liquid fraction 
    Lf=188000 # Lf latent heat of fusion


    # Material properties
    L=0.1# m
    k=0.9 # material thermal resistivity (1/lambda)
    rho=800 # density
    
    Tmin=20
    Tmax=30
    T_init=Tmin
    
    n = 12
    K = init_K(n)
    dx=L/n #

    dt = 5
    T_plus,Cp_t=np.zeros(n),np.zeros(n)
    T=np.ones(n)*T_init # initialize at Tmin
    
    # simulation time and time step
    t=0
    hours=0.1
    sim_time=hours*3600 #s
    
    Fo = generate_Fo(n,dt,T_init,dTf,Tf)
    
    while t < sim_time:
        # boundary conditions
        T[0]=Tmin
        T[n-1]=Tmax
        
        # update T 
        T_plus=Fo*np.dot(K,T) + T
        
        # update the local apparent Cp
        for i in range(n):
            Cp_t[i] = fc_Cp_apparent(T_plus[i],dTf,Tf,Lf,Cp_s,Cp_l)
        
        # Fo is updated and stability is tested
        Fo=k*dt/(rho*Cp_t*dx**2)
        print(Fo)
        if len(np.argwhere(Fo > 0.5))>0 :
            indice=np.argwhere(Fo > 0.5)
            print("stability issue during simulation... Fo =", Fo[indice[0]])
            print("occurred for... t=", round(t,1), " / i=")
        t+=dt #time increment
        T=T_plus # T turns into T_plus to allow the calculation of the next T_plus
    
    # after the simulation is terminated, T is plotted with Tf as a reference
    x_pos=np.arange(0,L,dx)
    plt.plot(x_pos, np.ones(n)*Tf,color=colors[-1],linestyle="-",alpha=0.9,marker='')
    plt.xlabel("x position [m]")
    plt.ylabel("Temperature [°C]")
    plt.plot(x_pos, T_plus, alpha=0.65, linestyle="--",marker='')



### Interactive plot

Tf and dTf can be picked in this plot. 

In [45]:
%matplotlib inline
interact_manual(compute_and_plot_PCM, Tf = widgets.FloatSlider(value=27,    min=20,
                                               max=30,
                                               step=0.1),
                dTf = widgets.FloatSlider(value=0.01,    min=0.001,
                                               max=0.1,
                                               step=0.001)

               )


interactive(children=(FloatSlider(value=27.0, description='Tf', max=30.0, min=20.0), FloatSlider(value=0.01, d…

<function __main__.compute_and_plot_PCM(Tf, dTf)>